In [2]:
import os
import pandas as pd
import sqlite3
import contextlib

In [3]:
database_path = "/data/s202633/X-finder/data/database/photorhabdus_xenorhabdus_old.db"
conn = sqlite3.connect(database_path)

In [4]:
sql = """
    SELECT sublists.ROWID AS sublistID,
            h.host_type, 
            h.file,
            h.organism,
            s.seq_accession, 
            s.description,
            MIN(p.pfam_start),
            MAX(p.pfam_end)
        FROM sublists
            INNER JOIN pfams AS p
                ON pfamID BETWEEN first_pfamID AND last_pfamID
            INNER JOIN seq_records AS s
                ON p.seq_accession = s.seq_accession
            INNER JOIN hosts AS h 
                ON s.hostID = h.hostID
        WHERE clusterID = ?
    GROUP BY sublistID
    """
df = pd.read_sql_query(sql, conn, params=(200,))
df

,sublistID,host_type,file,organism,seq_accession,description,MIN(p.pfam_start),MAX(p.pfam_end)
0,13321,query,GCF_000196475.1_ASM19647v1_genomic.gbk,Photorhabdus asymbiotica,NC_012962.1,"Photorhabdus asymbiotica, complete sequence",2474346,2490769
1,13322,ref,NBC_01231.gbk,Streptomyces cellulosae_0.8997 Unclassified.,NBC_01231_1,Streptomyces cellulosae_0.8997 strain NBC_01231,3795182,3804904


In [14]:
sql = '''
    SELECT c.ROWID as cds_rowid,
    c.seq_accession,
            c.locus_tag, 
            p.pfam_num, 
            p.antismash_core,
            p.pfam_start,
            p.pfam_end
        FROM sublists
            INNER JOIN pfams AS p
                ON pfamID BETWEEN first_pfamID AND last_pfamID
            INNER JOIN cds AS c
                ON p.locus_tag = c.locus_tag
        WHERE sublists.ROWID = ?
        '''
df = pd.read_sql_query(sql, conn, params=(13321,))
df["pfams"] = list(zip(df["pfam_num"], df["antismash_core"]))
df

,cds_rowid,seq_accession,locus_tag,pfamID,pfam_num,antismash_core,pfam_start,pfam_end,pfams
0,2931639,NC_012962.1,PAU_RS10680,3289479,107,0,2474346,2474799,"(107, 0)"
1,2931639,NC_012962.1,PAU_RS10680,3289480,8240,0,2474934,2475291,"(8240, 0)"
2,2931640,NC_012962.1,PAU_RS10685,3289481,7690,0,2475795,2476416,"(7690, 0)"
3,2931641,NC_012962.1,PAU_RS10690,3289482,13847,0,2477396,2477714,"(13847, 0)"
4,2931642,NC_012962.1,PAU_RS10695,3289483,975,0,2477859,2478591,"(975, 0)"
5,2931642,NC_012962.1,PAU_RS10695,3289484,550,0,2478666,2478858,"(550, 0)"
6,2931642,NC_012962.1,PAU_RS10695,3289485,501,0,2479191,2480313,"(501, 0)"
7,2931642,NC_012962.1,PAU_RS10695,3289486,668,0,2480388,2481609,"(668, 0)"
8,2931643,NC_012962.1,PAU_RS10700,3289487,550,1,2481831,2482011,"(550, 1)"
9,2931643,NC_012962.1,PAU_RS10700,3289488,8659,1,2482239,2482812,"(8659, 1)"


In [12]:
locus_tag = "PAU_RS10680"
list(df[df["locus_tag"]==locus_tag]["pfam_num"])

['107', '8240']

In [45]:
class Cds:
    
    def __init__(self, locus_tag, product, core_genome):
        self.locus_tag = locus_tag
        self.product = product
        self.core_genome = core_genome
    
    @classmethod
    def _from_row(cls, row):
        cds = cls(*row)
        return cds
    

with contextlib.closing(conn.cursor()) as c:
    sql = """
        SELECT locus_tag, product, core_genome
        FROM cds
        WHERE ROWID BETWEEN 2931639 AND 2931644
              AND seq_accession = ? 
        """
    rows = c.execute(sql, ("NC_012962.1",)).fetchall()
all_cds = [Cds._from_row(row) for row in rows]
all_cds

In [51]:
for cds in all_cds:
    print(cds.product, cds.core_genome)

crotonyl-CoA carboxylase/reductase 0
MFS transporter 0
class I SAM-dependent methyltransferase 0
AMP-binding protein 0
type I polyketide synthase 0
AMP-binding protein 1


In [54]:
dict([(cds.product, cds.core_genome) for cds in all_cds])

{'crotonyl-CoA carboxylase/reductase': 0,
 'MFS transporter': 0,
 'class I SAM-dependent methyltransferase': 0,
 'AMP-binding protein': 1,
 'type I polyketide synthase': 0}

In [53]:
[(cds.product, cds.core_genome) for cds in all_cds]

[('crotonyl-CoA carboxylase/reductase', 0),
 ('MFS transporter', 0),
 ('class I SAM-dependent methyltransferase', 0),
 ('AMP-binding protein', 0),
 ('type I polyketide synthase', 0),
 ('AMP-binding protein', 1)]

In [ ]:
[x for b in a for x in b]

In [50]:
cds_in_cluster_dict = dict()
for cds in all_cds:
    if cds.product in cds_in_cluster_dict.keys():
        cds_in_cluster_dict[cds.product].add(cds.core_genome)
    else: 
        cds_in_cluster_dict[cds.product] = set([cds.core_genome])
cds_in_cluster_dict

{'crotonyl-CoA carboxylase/reductase': {0},
 'MFS transporter': {0},
 'class I SAM-dependent methyltransferase': {0},
 'AMP-binding protein': {0, 1},
 'type I polyketide synthase': {0}}

In [46]:
for cds in all_cds:
    # list of (pfam_num, antismash_core_binary)
    cds.pfams = list(df[df["locus_tag"]==cds.locus_tag]["pfams"])
    print(cds.locus_tag, cds.core_genome, cds.pfams)

PAU_RS10680 0 [('107', 0), ('8240', 0)]
PAU_RS10685 0 [('7690', 0)]
PAU_RS10690 0 [('13847', 0)]
PAU_RS10695 0 [('975', 0), ('550', 0), ('501', 0), ('668', 0)]
PAU_RS10700 0 [('550', 1), ('8659', 1), ('698', 1), ('2801', 1), ('109', 1)]
PAU_RS10705 1 [('550', 0), ('501', 0), ('668', 0), ('550', 0), ('501', 0)]


In [34]:
for cds in all_cds:
    print(cds.locus_tag, cds.pfams)

PAU_RS10680 [('107', 0), ('8240', 0)]
PAU_RS10685 [('7690', 0)]
PAU_RS10690 [('13847', 0)]
PAU_RS10695 [('975', 0), ('550', 0), ('501', 0), ('668', 0)]
PAU_RS10700 [('550', 1), ('8659', 1), ('698', 1), ('2801', 1), ('109', 1)]
PAU_RS10705 [('550', 0), ('501', 0), ('668', 0), ('550', 0), ('501', 0)]


In [55]:
myset = {0, 1, None}
if [1,0] in myset